In [19]:
import matplotlib.pyplot as plt
import pandas as pd
import torch

In [20]:
df = pd.read_csv("Gemini_BTCUSD_1h.csv", parse_dates=["date"])

In [21]:
def generate_label_window(closings, dim: int, lookahead: int):
    window = closings[:dim]
    window_last_price = window[-1]
    with_lookahead = closings[:dim + lookahead]

    stddev_half = torch.std(with_lookahead) * 0.5

    buy = torch.abs(torch.min(with_lookahead) - window_last_price) < stddev_half 
    sell = torch.abs(torch.max(with_lookahead) - window_last_price) < stddev_half

    if buy == sell:
        return 0
    
    return -1 if sell else 1

In [24]:
closings = torch.flip(torch.tensor(df["close"][:4000]), dims=(0,))
dim = 48
lookahead = 24
win_size = dim + lookahead

n = len(closings) - dim - lookahead

xTr_sell = []
yTr_sell = []

xTr_buy = []
yTr_buy = []

for win_start in range(n):
  xTr_i = closings[win_start:][:win_size]
  yTr_i = generate_label_window(xTr_i, dim, lookahead)
  
  if yTr_i == 1:
    xTr_buy.append(xTr_i)
    yTr_buy.append(1)
  elif yTr_i == -1:
    xTr_sell.append(xTr_i)
    yTr_sell.append(1)
  else:
    xTr_buy.append(xTr_i)
    yTr_buy.append(0)
    xTr_sell.append(xTr_i)
    yTr_sell.append(0)

print(xTr_sell, yTr_sell)

xTr_sell = torch.stack(xTr_sell)
xTr_buy = torch.tensor(xTr_buy)

yTr_sell = torch.stack(yTr_sell)
yTr_buy = torch.tensor(yTr_buy)

[tensor([28732.4500, 28880.6900, 28989.7000, 28955.0200, 29961.1700, 29841.4700,
        29687.4000, 29816.5800, 29782.0800, 29722.3800, 29729.1800, 27884.6600,
        28421.9900, 28695.9900, 28296.4100, 28421.9100, 29227.8400, 28740.0600,
        29043.8800, 28888.3200, 29097.1500, 29083.0600, 28800.4400, 28899.2000,
        28954.0200, 28943.0000, 28970.5000, 28936.9700, 28862.2000, 28918.7700,
        28961.0300, 29144.5900, 29135.7500, 29288.2500, 29761.4600, 29655.8600,
        29629.9900, 29654.4800, 29396.2100, 29479.0500, 29555.0000, 29527.9300,
        29408.0200, 29409.0600, 29493.0100, 29487.6900, 29466.1300, 29498.3500,
        29262.7600, 29205.8100, 29262.1300, 29247.2100, 29234.6700, 29035.4000,
        29088.6300, 29156.5600, 29279.8800, 29251.9500, 29248.7700, 29336.2800,
        29355.9600, 29363.7700, 29371.0900, 29341.7800, 29276.5000, 29304.5400,
        29404.4300, 29377.5100, 29395.9500, 29385.2100, 29342.1700, 29366.9800],
       dtype=torch.float64), tensor([2

ValueError: only one element tensors can be converted to Python scalars

In [ ]:
import torch

model = torch.nn.Sequential( 
    torch.nn.Linear(in_features = dim, out_features = dim), 
    torch.nn.Softmax(dim=1),
    torch.nn.Linear(in_features = dim, out_features = 1), 
    torch.nn.Softmax(dim=1),
)

# Train the model 
criterion = torch.nn.CrossEntropyLoss() 
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)
  
num_epochs = 1000
for epoch in range(num_epochs): 
    # Forward pass 
    y_pred = model(xTr_buy) 
    loss = criterion(y_pred, yTr_buy) 
  
    # Backward pass and optimization 
    optimizer.zero_grad() 
    loss.backward() 
    optimizer.step() 
  
    # Print the loss every 100 epochs 
    if (epoch+1) % 100 == 0: 
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


3660 3509
